In [1]:
!python -m spacy download ru_core_news_sm

^C


In [2]:
import spacy
import pandas as pd

from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_sm")

In [3]:
df_train = pd.read_csv('train.csv', encoding="utf8")
df_val = pd.read_csv('valid.csv', encoding="utf8")

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.0/15.3 MB 330.3 kB/s eta 0:00:47
     --------------------------------------- 0.0/15.3 MB 326.8 kB/s eta 0:00:47
     --------------------------------------- 0.1/15.3 MB 409.6 kB/s eta 0:00:38
     --------------------------------------- 0.1/15.3 MB 654.9 kB/s eta 0:00:24
      -------------------------------------- 0.2/15.3 MB 981.9 kB/s eta 0:00:16
      --------------------------------------- 0.4/15.3 MB 1.3 MB/s eta 0:00:12
     - -------------------------------------- 0.6/15.3 MB 1.9 MB/s eta 0:00:08
     - -------------------------------------- 0.6/15.3 MB 1.9 MB/s eta 0:00:08
     --- ------------------------------------ 1.4/15.3 MB 3.4 MB/s eta 0:00:05
     ---- ----------------------------------- 1.7/15.3 MB 3.7 MB/s eta 0:00:04
     ---- ----------------------------------- 1.9/15.3 MB 3.9 MB/s eta 0:00:04
     ---- ----------------------------------- 1.9/15.3

In [4]:
df_train = df_train.sample(frac=0.5)
df_val = df_val.sample(frac=0.5)

In [5]:
df_train

,Unnamed: 0,text,sentiment
17398,38496,Удаление зуба мудрости Удалял зуб мудрости на ...,1
57421,78519,Очень уютный и комфортный отель и большая зеле...,1
156913,178011,"Новый,чистый,приятный, хорошие завтраки, бассе...",1
47633,68731,Снятие инвалидности Моему ребенку сняли инвали...,2
20849,41947,Я как то спустилась открыть а мне мужик попыта...,0
...,...,...,...
26186,47284,В моей жизни Кашкай появился в июне...,1
144897,165995,"Ну тык разные кораблики нужны, разные кораблик...",1
89726,110824,Комитет по регулированию естественных монополи...,0
65838,86936,Довольно часто коэффициент Джини приводится бе...,1


In [6]:
df_train['sentiment'].value_counts()

sentiment
1    45189
2    24962
0    24795
Name: count, dtype: int64

In [7]:
df_train = df_train[df_train['sentiment'] != 0]
df_val = df_val[df_val['sentiment'] != 0]

In [8]:
train_data = [tuple(df_train.iloc[i].values)[1:] for i in range(df_train.shape[0])]
valid_data = [tuple(df_val.iloc[i].values)[1:] for i in range(df_val.shape[0])]

In [9]:
train_data

[('Удаление зуба мудрости Удалял зуб мудрости на Войковской, у Шульгина Романа Валерьевича. Удаление согласовали по страховке. Удалили быстро и аккуратно. Рана потом почти не болела и зажила быстро. ',
  1),
 ('Очень уютный и комфортный отель и большая зеленая территория на которой живут зайцы. Вид на залив. Бесплатная парковка и платный охраняемый паркинг. Трамвай до центра города в 3 минутах ходьбы от отеля. Отличные рестораны. Завтрак очень разнообразен и вкусен.',
  1),
 ('Новый,чистый,приятный, хорошие завтраки, бассейн есть. Номера хорошие светлый. Рекомендую.Пляж рядом.Цены умеренные. Жили там неделю, остались очень довольны по сравнению с другими отелями в Пхукете. До Бангла-Роуд всего минут пешком, много магазинов рядом.',
  1),
 ('Снятие инвалидности Моему ребенку сняли инвалидность. Объяснили так, что у него незначительные нарушения. Диагноз ребенку поставили такой: асептический некроз эпифиза правой большеберцовой кости. Наблюдаемся у профессора, доктора медициниских наук, 

In [10]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label==1:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        else:
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        # we need to set the (text)cat(egory) for each document
        #doc.cats["positive"] = label
        # put them into a nice list
        docs.append(doc)
    return docs

In [11]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short.
# In practice take as much data as you can get.
# you can always reduce it to make the script even faster.
num_texts = 5000
# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])

100%|██████████| 5000/5000 [01:25<00:00, 58.52it/s]


In [12]:

# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 5000/5000 [01:28<00:00, 56.23it/s]


на этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг (textcat), копируем его руками(!) в base_config.cfg, указываем правильные пути до трейн и вэлид


In [20]:
!python -m spacy train base_config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       39.58    0.40
  0     200         48.35       40.74    0.41
  0     400         45.20       70.65    0.71
  0     600         45.42       67.56    0.68
  0     800         37.58       67.96    0.68
  0    1000         40.34       74.42    0.74
  0    1200         40.47       64.26    0.64
  0    1400         32.10       71.15    0.71
  0    1600         35.82       78.54    0.79
  0    1800         26.84       78.37    0.78
  0    2000         37.20       76.89    0.77
  0    2200         27.66       70.34    0.70
  1    2400         24.04       80.72    0.81
  1    2600         23.26   

In [2]:
import spacy
# load thebest model from training
nlp = spacy.load("./output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
{'positive': 0.06642790883779526, 'negative': 0.9335720539093018}
the sentiment is negative
{'positive': 0.9999839067459106, 'negative': 1.6055362721090205e-05}
the sentiment is positive
{'positive': 1.0, 'negative': 1.5687853540960554e-16}
the sentiment is positive
{'positive': 7.057777665343437e-26, 'negative': 1.0}
the sentiment is negative
